In [3]:
from datasets import load_dataset
dataset=load_dataset('cnn_dailymail','3.0.0')
print(dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})


In [4]:
print(dataset['train'][0])
print(dataset['train'][1])
print(dataset['train'][2])

{'article': 'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office char

In [5]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [6]:
tokenizer = T5Tokenizer.from_pretrained("t5-small")


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [7]:
print(dataset['train'].features)

{'article': Value(dtype='string', id=None), 'highlights': Value(dtype='string', id=None), 'id': Value(dtype='string', id=None)}


In [8]:
def preprocess_data(batch):

    inputs = tokenizer(
        batch["article"],
        max_length=512,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )


    labels = tokenizer(
        batch["highlights"],
        max_length=150,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )


    return {
        "input_ids": inputs["input_ids"].squeeze(),
        "attention_mask": inputs["attention_mask"].squeeze(),
        "labels": labels["input_ids"].squeeze()
    }


train_dataset = dataset["train"].map(preprocess_data, batched=True)
val_dataset = dataset["validation"].map(preprocess_data, batched=True)


Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

In [9]:
from transformers import T5ForConditionalGeneration

# Load the T5 model for conditional generation
model = T5ForConditionalGeneration.from_pretrained('t5-small')


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [10]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="no",
    learning_rate=5e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=1,
    logging_steps=100,
    report_to="none",
    fp16=True,
    gradient_accumulation_steps=2,
    dataloader_num_workers=4,
    save_steps=1000,
    load_best_model_at_end=False,
)


In [11]:
from transformers import Trainer


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [12]:

trainer.train()


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Step,Training Loss
100,2.833300
200,1.095600
300,1.067100
400,1.039200
500,1.043000
600,1.038400
700,1.016600
800,1.034400
900,1.027900
1000,1.022800


TrainOutput(global_step=4486, training_loss=1.055265454697492, metrics={'train_runtime': 6264.8273, 'train_samples_per_second': 45.829, 'train_steps_per_second': 0.716, 'total_flos': 3.885717256981709e+16, 'train_loss': 1.055265454697492, 'epoch': 0.9998885545525465})

In [13]:
custom_input = """Artificial intelligence (AI) has rapidly evolved over the past decade, impacting various industries such as healthcare, finance, education, and transportation.
From automating repetitive tasks to providing personalized recommendations, AI technologies are transforming how we live and work.
As companies and governments invest more in AI research and development, questions around ethics, privacy, and job displacement continue to emerge.
The future of AI holds both great promise and significant challenges, requiring careful consideration of its societal implications."""

inputs = tokenizer(
    custom_input,
    max_length=512,
    truncation=True,
    padding="max_length",
    return_tensors="pt"
)

outputs = model.generate(
    inputs["input_ids"].to(model.device),
    max_length=150,
    num_beams=4,
    early_stopping=True
)

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated Output:", generated_text)


Generated Output: AI has rapidly evolved over the past decade, impacting healthcare, finance, education, transportation. As companies and governments invest more in AI research and development, questions about ethics, privacy, and job displacement continue to emerge.
